In [1]:
import os
import sys
import math
import argparse
import clip
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

sys.path.append(os.path.dirname(os.getcwd()))

from dalle.models import Dalle
from dalle.utils.utils import set_seed, clip_score

device = 'cuda:0'
model = Dalle.from_pretrained(r"D:\pyproject\mindall-e\checkpoints")
model_clip, preprocess_clip = clip.load("ViT-B/32", device=device)

model_clip.to(device=device)
model.to(device=device)

def sampling(prompt, top_k, softmax_temperature, seed, num_candidates=4, num_samples_for_display=4):
    # Setup
    n_row = int(math.sqrt(num_samples_for_display))
    n_col = int(math.sqrt(num_samples_for_display))
    set_seed(seed)
    
    # Sampling
    images = model.sampling(prompt=prompt,
                            top_k=top_k,
                            top_p=None,
                            softmax_temperature=softmax_temperature,
                            num_candidates=num_candidates,
                            device=device).cpu().numpy()
    images = np.transpose(images, (0, 2, 3, 1))

    # CLIP Re-ranking
    rank = clip_score(prompt=prompt, images=images, model_clip=model_clip, preprocess_clip=preprocess_clip, device=device)
    images = images[rank]
    
    images = images[:num_samples_for_display]
    fig = plt.figure(figsize=(8*n_row, 8*n_col))

    for i in range(num_samples_for_display):
        ax = fig.add_subplot(n_row, n_col, i+1)
        ax.imshow(images[i])
        ax.set_axis_off()

    plt.tight_layout()
    plt.show()

D:\pyproject\mindall-e\checkpoints\tokenizer successfully restored..


D:\pyproject\mindall-e\dalle\models\stage1\vqgan.py:91: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(path, map_location='cpu')['state_dict']


D:\pyproject\mindall-e\checkpoints\stage1_last.ckpt successfully restored..


D:\pyproject\mindall-e\dalle\models\stage2\transformer.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(path, map_location='cpu')['state_dict']


D:\pyproject\mindall-e\checkpoints\stage2_last.ckpt succesfully restored..


In [2]:
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

output = widgets.Output()
plot_output = widgets.Output()

def btn_eventhandler(obj):
    output.clear_output()
    plot_output.clear_output()
    
    with output:
        print(f'SEED: {slider_seed.value}')
        print(f'Softmax Temperature: {slider_temp.value}')
        print(f'Top-K: {slider_topk.value}')
        print(f'Text prompt: {wd_text.value}')
        
    with plot_output:
        sampling(prompt=wd_text.value, top_k=slider_topk.value, softmax_temperature=slider_temp.value, seed=slider_seed.value)
    
slider_seed = widgets.IntSlider(
    min=0,
    max=1024,
    step=1,
    description='RND SEED: ',
    value=0
)
slider_topk = widgets.IntSlider(
    min=0,
    max=512,
    step=16,
    description='TOP-K:',
    value=256
)
slider_temp = widgets.FloatSlider(
    min=0.0,
    max=5.0,
    step=0.2,
    description='SOFTMAX TEMPERATURE:',
    value=1.0
)
wd_text = widgets.Text(
    value='A painting of a monkey with sunglasses in the frame',
    placeholder='Text prompt',
    description='String:',
    disabled=False
)

display(slider_seed)
display(slider_temp)
display(slider_topk)
display(wd_text)

btn = widgets.Button(description='Generate!')
display(btn)
btn.on_click(btn_eventhandler)

display(output)
display(plot_output)

IntSlider(value=0, description='RND SEED: ', max=1024)

FloatSlider(value=1.0, description='SOFTMAX TEMPERATURE:', max=5.0, step=0.2)

IntSlider(value=256, description='TOP-K:', max=512, step=16)

Text(value='A painting of a monkey with sunglasses in the frame', description='String:', placeholder='Text pro…

Button(description='Generate!', style=ButtonStyle())

Output()

Output()